In [8]:
%load_ext autoreload
%autoreload 2
import torch

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load model/data

In [2]:
from datasets.audioset import Audioset
from torchvision import transforms
from torch.utils.data import DataLoader

audio_set_val = Audioset(split_mode='validation', fixed_padding=True)

In [3]:
from classification.models.SpectrogramCNN import SpectrogramCNN

state_dict_path = '/nfs/students/summer-term-2020/project-4/data/models/SpectrogramBasedCNN.pt'

model = SpectrogramCNN()
model.load_state_dict(torch.load(state_dict_path))
model = model.cuda()
model = model.eval()

In [109]:
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN

model_state_dict_path = "/nfs/students/summer-term-2020/project-4/yan/models/best_model_state_dict.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model = model.eval()

## FGSM

In [59]:
from attacks.fgsm import FGSM

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'epsilon': 0.2}

fgsm = FGSM(model, valid_loader, attack_parameters)
fgsm.attack()

100%|██████████| 1687/1687 [00:20<00:00, 82.01it/s]


In [ ]:
# fgsm.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [52]:
fgsm.report()

Attack-Parameters:	{'epsilon': 0.2}
Early stopping: 	False (-1)

Successfully attacked:	262
Total attacked: 	1483
Total processed:	1687

Success-Rate: 		0.18
Perturbed Accurracy: 	0.72


## PGD

In [57]:
from attacks.pgd import PGD

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'epsilon': 0.5, 'num_iter': 30}

pgd = PGD(model, valid_loader, attack_parameters, early_stopping=20)
pgd.attack()

  3%|▎         | 43/1687 [00:09<06:04,  4.51it/s]

Early stopping


In [ ]:
# pgd.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [58]:
pgd.report()

Attack-Parameters:	{'epsilon': 0.5, 'num_iter': 30}
Early stopping: 	True (20)

Successfully attacked:	20
Total attacked: 	40
Total processed:	44

Success-Rate: 		0.5
Perturbed Accurracy: 	0.45


## Volume attacks

In [110]:
from attacks.volume import VolumeAttack

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'epsilon': 0.1, 'num_iter': 10}

vol = VolumeAttack(model, valid_loader, attack_parameters, early_stopping=3)
vol.attack()

  6%|▌         | 97/1687 [00:55<15:14,  1.74it/s]

Early stopping


In [112]:
# vol.showAdversarialExample(target_class=1)

In [63]:
vol.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30}
Early stopping: 	False (-1)

Successfully attacked:	720
Total attacked: 	1483
Total processed:	1687

Success-Rate: 		0.49
Perturbed Accurracy: 	0.45


## Interpolation attack

In [138]:
import librosa
path_tum_sound = '/nfs/students/summer-term-2020/project-4/yan/tum.wav'
tum_sound,sr = librosa.load(path_tum_sound, sr=48000)

max_length = len(audio_set_val[0][0])
padding = int((max_length - len(tum_sound))/2)
zero_padded_data = torch.zeros(max_length)
zero_padded_data[padding:padding+tum_sound.shape[0]] = torch.from_numpy(tum_sound)
tum_sound = zero_padded_data

In [ ]:
from attacks.interpolation import InterpolationAttack

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tum_sound}

ia = InterpolationAttack(model, valid_loader, attack_parameters, early_stopping=1)
ia.attack()

In [298]:
# ia.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [297]:
ia.report()

Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 30, 'overlay_sound': tensor([0., 0., 0.,  ..., 0., 0., 0.])}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	67
Total processed:	75

Success-Rate: 		0.01
Perturbed Accurracy: 	0.88


## Time stretching attack

In [187]:
from attacks.speed import TimeStretchAttack

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'num_iter': 30, 'lower': 0.1, 'upper': 0.5}

tsa = TimeStretchAttack(model, valid_loader, attack_parameters, early_stopping=5)
tsa.attack()

  8%|▊         | 128/1687 [00:28<05:46,  4.49it/s]

Early stopping


In [197]:
tsa.showAdversarialExample(target_class=0)

In [170]:
tsa.report()

Attack-Parameters:	{'num_iter': 30, 'lower': 0.1, 'upper': 1}
Early stopping: 	True (20)

Successfully attacked:	16
Total attacked: 	1483
Total processed:	1687

Success-Rate: 		0.01
Perturbed Accurracy: 	0.87


## Pitch attack

In [27]:
from attacks.attack import Attack

from torchaudio import functional as AF
import torch.nn.functional as F
import torchaudio
import math
import torch

class PitchAttack(Attack):

    def attackSample(self, x, y, num_iter=1, lower=1, upper=5):
        n_steps_search_range = torch.arange(lower, upper, (upper-lower)/num_iter)
        losses = []
        stretched_inputs = []
        
        with torch.no_grad():
            for n_steps in n_steps_search_range:
                stretched = self.pitch_shift(x[0].squeeze(), sr=x[1], n_steps=n_steps)
                stretched = stretched.unsqueeze(0)
            stretched_inputs.append(stretched)
            losses.append(F.nll_loss(self.model([stretched, x[1]]), y))
        best_rate = torch.stack(losses).argmax().item()
        return stretched_inputs[best_rate].clamp(-1,1), x[1]

    def pitch_shift(self, sample, sr, n_steps, bins_per_octave=12): 
        # https://librosa.github.io/librosa/_modules/librosa/effects.html#pitch_shift
        assert bins_per_octave >= 1
        rate = 2.0 ** (-float(n_steps) / bins_per_octave)

        # Stretch in time, then resample, compare librosa
        resample = torchaudio.transforms.Resample(float(sr.cpu())/rate, sr.cpu()).cpu()
        y_shift = resample(self.time_stretch(sample, rate).cpu()).cuda() # not diff'able
        
        # back to original size
        max_length = sample.shape[0]
        low = int((y_shift.shape[0] - max_length)/2)
        return y_shift[low:low+max_length]

    """
        Time stretching *without* padding
        
        Note this function is **not** differentiable:
        through speedup frames get deleted/inserted, 
        which makes the function not differentiable)
    """
    def time_stretch(self, sample, speedup_rate):
        if speedup_rate == 1:
            return sample

        n_fft = torch.tensor(2048)  # windowsize
        hop_length = torch.floor(n_fft / 4.0).int().item()

        # time stretch
        stft = torch.stft(sample, n_fft.item(), hop_length=hop_length).unsqueeze(0)
        phase_advance = torch.linspace(0, math.pi * hop_length, stft.shape[1])[..., None].cuda()
        # time stretch via phase_vocoder (not differentiable):
        vocoded = AF.phase_vocoder(stft, rate=speedup_rate, phase_advance=phase_advance) 
        return AF.istft(vocoded, n_fft.item(), hop_length=hop_length).squeeze()

In [ ]:
#from attacks.pitch import PitchAttack

valid_loader = DataLoader(audio_set_val, batch_size=1)
attack_parameters = {'num_iter': 5}

pa = PitchAttack(model, valid_loader, attack_parameters, early_stopping=1)
pa.attack()

  0%|          | 3/1687 [04:18<40:22:02, 86.30s/it]

In [ ]:
pa.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

In [56]:
pa.report()

Attack-Parameters:	{'num_iter': 5, 'sr': 48000}
Early stopping: 	True (1)

Successfully attacked:	1
Total attacked: 	10
Total processed:	10

Success-Rate: 		0.1
Perturbed Accurracy: 	0.9
